In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set()
sns.set(rc={"xtick.bottom" : True, "ytick.left" : True})
plt.rcParams['axes.facecolor'] = 'ebe8f2'

%matplotlib inline

color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
def read(path):
    df = pd.read_csv(path)
    df = df.sort_values(by=["rate", "start"])
    df['latency'] = df['latency'] * 1000
    return df

def latency_scatter(dfs):
    """Create a scatter plot with all measured latencies"""
    for name, df in dfs.items():
        plt.scatter(df['rate'], df['latency'], alpha=0.01, marker='.', label=name)
    plt.xlabel("Req/s")
    plt.ylabel("Latency (s)")
    plt.legend()

def quantile_plot(dfs, y_max=400, q_low=0.2, q_high=0.8):
    """Create a quantile plot with a line for the median and a filled area for a quantile range"""
    f, a = plt.subplots(figsize=(12,8), dpi= 100)
    handles = []
    names = []
    color_iter = iter(color_cycle)
    for name, df in dfs.items():
        color = next(color_iter)
        grouped_latency = df.groupby("rate")["latency"]
        q20 = grouped_latency.quantile(q_low)
        q50 = grouped_latency.quantile(0.5)
        q80 = grouped_latency.quantile(q_high)
        fill = a.fill_between(q20.index, q20, q80, alpha=0.2, color=color)
        med = a.plot(q50.index, q50, color=color)
        dummy_fill = a.fill(np.NaN, np.NaN, alpha=0.2, color=color)
        handles.append((med[0], dummy_fill[0]))
        names.append(name)
    a.set_xlabel("Rate (requests/s)", fontsize=14)
    a.set_ylabel("Latency (ms)", fontsize=14)
    a.set_ylim([0, y_max])
    a.legend(handles, names)
    
def rate_consistency(dfs):
    """Print request rate consistency. This measures the qualtiy of the test."""
    for name, df in dfs.items():
        starts = df.sort_values("start")[["rate", "start"]]
        diffqs = {}
        for rate in df['rate'].unique():
            diff = df[df["rate"] == rate].sort_values("start")["start"].diff()
            q = diff.quantile(0.5)
            diffqs[rate] = (round(q, 5), round(1/rate, 3), round(abs(q - 1/rate), 5))
        plt.plot(diffqs.keys(), [x[-1] for x in diffqs.values()], label=name)
    plt.title("Deviation from ideal request frequency")
    plt.xlabel("Rate (requests/s)")
    plt.ylabel("Deviation (s)")

In [ ]:
w1 = {
    "no batching": read("v100-w1"), 
    "t = 25ms": read("v100-b-t25-w1"),
    "t = 50ms": read("v100-b-t50-w1"), 
    "t = 100ms": read("v100-b-t100-w1"),
    "t = 200ms": read("v100-b-t200-w1"),
}
quantile_plot(w1)
plt.title("Median Latency of ResNet50 on V100", fontsize=18);